# Czech ULMFiT from scratch

This notebook is based on [Turkish ULMFiT from scratch](https://github.com/fastai/course-nlp/blob/master/nn-turkish.ipynb) notebook from [fast.ai](http://fast.ai) [NLP course](https://github.com/fastai/course-nlp/).

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [27]:
# GPU test
#import torch
#torch.cuda.current_device()
#torch.cuda.device(0)
#torch.cuda.device_count()
#torch.cuda.get_device_name(0)

'Tesla P4'

In [2]:
bs=128
data_path = Config.data_path()

lang = 'cs'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

In [3]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
lm_fns = [mdl_path/f'{lang}_wt', mdl_path/f'{lang}_wt_vocab']

## Czech wikipedia model

In [4]:
from nlputils import split_wiki,get_wiki

In [5]:
get_wiki(path, lang)
!head -n4 {path}/{name}

unzipping...
extracting...
<doc id="10" url="https://cs.wikipedia.org/wiki?curid=10" title="Astronomie">
Astronomie

Astronomie, řecky αστρονομία z άστρον (astron) hvězda a νόμος (nomos) zákon, česky též hvězdářství, je věda, která se zabývá jevy za hranicemi zemské atmosféry. Zvláště tedy výzkumem vesmírných těles, jejich soustav, různých dějů ve vesmíru i vesmírem jako celkem.


In [5]:
dest = split_wiki(path, lang)

/home/jupyter/.fastai/data/cswiki/docs already exists; not splitting


[Czech language](https://en.wikipedia.org/wiki/Czech_language) has declinations, so it needs special care!


In [11]:
data = (TextList.from_folder(dest, processor=[OpenFileProcessor(), SPProcessor(max_vocab_sz = 30000)])
        .split_by_rand_pct(0.1, seed=42)
        .label_for_lm()
        .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

(30000, 97506)

In [12]:
data = load_data(dest, f'{lang}_databunch', bs=bs)

In [13]:
data.show_batch()

idx,text
0,"▁v ▁severní ▁části ▁xxmaj ▁polska , ▁ve ▁vojvodství ▁xxmaj ▁po mor ski e , ▁poblíž ▁obcí ▁xxmaj ▁ ch oj nice , ▁xxmaj ▁brus y , ▁xxmaj ▁char zy kow y ▁v ▁srdci ▁xxmaj ▁tu ch ol ských ▁lesů , ▁největším ▁zalesněné m ▁území ▁v ▁xxmaj ▁polsku . ▁xxmaj ▁správa ▁parku ▁je ▁v ▁obci , ▁která ▁se ▁nachází ▁3 ▁km ▁od ▁xxmaj ▁ ch oj nic . ▁xxmaj ▁nachází ▁se"
1,"lon ▁( st . ▁xxmaj ▁peter sburg , ▁xxup ▁usa ). ▁xxmaj ▁je ▁deseti násobný m ▁mistrem ▁xxup ▁čr ▁v ▁olympijské m ▁tri at lon u . ▁xxmaj ▁při ▁svém ▁prvním ▁startu ▁na ▁dlouhé m ▁tri at lon u ▁- ▁xxmaj ▁iron man ▁xxmaj ▁florida ▁2013 ▁obsadil ▁3. ▁místo ▁a ▁časem ▁7 : 58 : 44 ▁se ▁zařadil ▁mezi ▁ne ce lou ▁dva cí tku ▁atlet ů ▁světa , ▁kteří"
2,▁páni ▁z ▁xxmaj ▁ne č tin . ▁xxmaj ▁nový ▁majitel ▁pro ▁město ▁roku ▁14 16 ▁získal ▁právo ▁pořád at ▁výroční ▁trh . ▁xxmaj ▁jindřich ▁nepře síd lil ▁na ▁ žlut ický ▁hrad . ▁xxmaj ▁jako ▁katolík ▁žil ▁až ▁do ▁konce ▁života ▁v ▁neu stál ých ▁bojích ▁s ▁ husit y . ▁xxmaj ▁roku ▁1421 ▁byly ▁xxmaj ▁ žlut ice ▁obsazen y ▁křižáck ým ▁vojskem . ▁xxmaj ▁praž ané ▁město ▁o
3,"▁festivalu , ▁přibyl o ▁oblíbený ch ▁xxmaj ▁ bus ▁xxmaj ▁partie s , ▁na bit ý ▁program ▁nabídl ▁již ▁podruhé ▁v ▁řadě ▁xxup ▁ bec ▁stan . ▁xxmaj ▁festival ▁pak ▁dále ▁pokračuje ▁ve ▁využívání ▁bez ho t ovost ní ▁systému ▁plat eb ▁a ▁rozšiřuje ▁ekologické ▁aktivity ▁v ▁rámci ▁programu ▁xxmaj ▁ roll ▁xxmaj ▁in ▁xxmaj ▁green . ▁xxmaj ▁oficiální ▁web ▁festivalu ▁xxmaj ▁oficiální ▁xxmaj ▁facebook ▁festivalu ▁xxmaj ▁oficiální ▁youtube ▁festivalu"
4,"▁jsou ▁garant ovány ▁po ▁dobu ▁20 ▁let ▁provozování ▁konkrétní ho ▁zařízení . ▁xxmaj ▁tzv . ▁zelený ▁bonus ▁se ▁však ▁mění ▁každý ▁rok ▁a ▁pro ▁konkrétní ▁výpočet ▁návrat nosti ▁je ▁potřeba ▁z o hled nit ▁cen ový ▁výnos ▁xxup ▁ eru ▁v ▁době ▁po řízení ▁foto vol ta ické ▁elektrárny . ▁xxmaj ▁pro ▁provozovatel e ▁je ▁jistě ▁zajímavá ▁i ▁skutečnost , ▁že ▁je ▁po ▁dobu ▁pěti ▁let ▁osvobozen ▁od ▁daní ▁z"


In [8]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.1, wd=0.1, pretrained=False).to_fp16()

In [6]:
lr = 3e-3
lr *= bs/48  # Scale learning rate by batch size

In [ ]:
# takes ~28 hours, therefore rather run this in terminal
# >>> lr = 3e-3
# >>> lr *= bs/48
# >>> learn.unfreeze()
# >>> learn.fit_one_cycle(12, lr, moms=(0.8,0.7))
# epoch     train_loss  valid_loss  accuracy  time
# 0         4.220986    4.310223    0.289666  2:20:18
# 1         4.138725    4.280440    0.289980  2:20:52
# 2         4.187871    4.383055    0.280299  2:21:02
# 3         4.170604    4.376703    0.281176  2:21:01
# 4         4.160329    4.325862    0.285679  2:21:03
# 5         4.060669    4.256678    0.292394  2:21:02
# 6         4.003880    4.159348    0.300838  2:21:05
# 7         3.907038    4.030453    0.313692  2:20:51
# 8         3.787868    3.886877    0.329324  2:20:52
# 9         3.651094    3.736353    0.346885  2:20:45
# 10        3.611639    3.603158    0.364177  2:20:39
# 11        3.561713    3.551184    0.371601  2:20:40

# learn.to_fp32().save(lm_fns[0], with_opt=False)
# learn.data.vocab.save(lm_fns[1].with_suffix('.pkl'))

epoch,train_loss,valid_loss,accuracy,time
0,4.261763,4.478383,0.271987,2:17:47
1,4.132481,4.315895,0.286108,2:18:05


## Czech sentiment analysis resources

http://liks.fav.zcu.cz/sentiment/

https://www.aclweb.org/anthology/W13-1609

### Language model

In [6]:
path_clas = path/'csfd'
path_clas.ls()

[PosixPath('/home/jupyter/.fastai/data/cswiki/csfd/positive.txt'),
 PosixPath('/home/jupyter/.fastai/data/cswiki/csfd/neutral.txt'),
 PosixPath('/home/jupyter/.fastai/data/cswiki/csfd/licence.txt'),
 PosixPath('/home/jupyter/.fastai/data/cswiki/csfd/cs_clas_databunch'),
 PosixPath('/home/jupyter/.fastai/data/cswiki/csfd/models'),
 PosixPath('/home/jupyter/.fastai/data/cswiki/csfd/negative.txt')]

In [7]:
pos = (path_clas/'positive.txt').open().readlines()
pos_df = pd.DataFrame({'text':pos})
pos_df['pos'] = 1
pos_df.head()

,text,pos
0,Jako se mě líbila stejně jako jednička. Má sic...,1
1,Sice se rika ze dvojky pokulhavaji za jednicko...,1
2,Tenhle film můžu opravdu kdykoliv. A to nejen ...,1
3,velice povedená krimikomedie s plejádou skvělý...,1
4,Rok 1969. Při výročí invaze střílejí do lidí č...,1


In [8]:
neg = (path_clas/'negative.txt').open().readlines()
neg_df = pd.DataFrame({'text':neg})
neg_df['pos'] = 0
neg_df.head()

,text,pos
0,Tommy Lee Jonesovi jsou role jako je tato šity...,0
1,se stejnou upřímností a váhou jako je Pravdivá...,0
2,Fakt blbost. 1. díl byl ucházející a dalo se n...,0
3,Jako mladíkovi uprostřed normalizačních sedmde...,0
4,...aneb proč neudělat ze Tří mušketyrů sci-fi....,0


In [9]:
df = pd.concat([pos_df,neg_df], sort=False)

In [15]:
data_lm = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

data_lm.save(f'{lang}_clas_databunch')

In [16]:
data_lm = load_data(path_clas, f'{lang}_clas_databunch', bs=bs)

In [17]:
data_lm.show_batch()

idx,text
0,"o . ▁a ▁o pet ▁nez klam ala ▁ perfekt ni ▁ muzik a ! ! ! ▁xxbos ▁xxmaj ▁ten hle ▁film ▁ m ů ž u ▁opravdu ▁kdykoliv . ▁a ▁to ▁nejen ▁díky ▁par fek tní mu , ▁ hlá ška mi ▁na bit ému ▁scénář i ▁a ▁herecký m ▁výkonů m ▁všech ▁z ů čas t ně ných ▁( hlavně ▁pak ▁trojice ▁xxmaj ▁troj an , ▁xxmaj ▁mit"
1,"▁před ▁sebou ▁2 ▁prázdné ▁krabic e ▁od ▁velkého ▁pop corn u , ▁ prázdn á ▁dvou lit rov ka ▁a ▁na ▁rukou ▁oko us ané ▁ne h ty . ▁xxmaj ▁to ▁co ▁jsem ▁ten krát ▁v ▁k ině ▁zažil , ▁byl ▁oprav dob ý ▁filmový ▁zážitek , ▁jaký ▁se ▁u ▁mě ▁zatím ▁ne opa koval . ▁xxmaj ▁celou ▁dobu ▁jsem ▁hl tal ▁ty ▁nádherné ▁záběry , ▁s trh ující ▁bitevní"
2,". ▁xxbos ▁ špi čkový ▁komerční ▁ válečný ▁seriál , ▁který ▁mě ▁do ▁druhé ▁světové ▁války ▁v táhl ▁snad ▁nejlépe ▁ze ▁všech ▁filmů ▁i ▁seriál ů ▁vůbec . ▁xxmaj ▁osudy ▁dotyčný ch ▁vojáků ▁se ▁nevy poč it atel ně ▁kříží , ▁až ▁to ▁ve ▁ m ne ▁mnohdy ▁vyvolá valo ▁deprese ▁a ▁pocity ▁úzkost i . ▁xxmaj ▁všechny ▁příběhy , ▁zřejmě ▁měly ▁vz bud it ▁v ▁di vá cích ▁odraz"
3,"a ▁xxmaj ▁me lich erová ), ▁které ▁už ▁propadl ▁ne jeden ▁muž ▁( mus ím ▁při zna t , ▁že ▁včetně ▁mě ), ▁ale ▁žádný ▁jí ▁zatím ▁nebyl ▁dost ▁dobrý . ▁xxmaj ▁ja šek ▁se ▁jí ▁ líb í , ▁ovšem ▁protože ▁je ▁příliš ▁mě k ký , ▁tak ▁nakonec ▁podléhá ▁pořád nému ▁chlap ovi , ▁který ▁se ▁nez d ráh á ▁ji ▁u ho dit , ▁muži ▁činu ▁a"
4,". . ▁xxbos ▁xxmaj ▁za ▁socialistické ho ▁xxmaj ▁československa ▁u ▁nás ▁běžel ▁tento ▁film ▁s ▁názvem ▁"" smrt ▁xxmaj ▁jo a ▁xxmaj ▁indián a "" ▁a ▁jako ▁deseti leté mu ▁kluk ovi ▁se ▁ mi ▁to ▁moc ▁líbil o . ▁xxmaj ▁je ▁to ▁vlastně ▁3. ▁díl ▁čtyř dílné ▁televizní ▁série ▁http : ▁/ ▁/ ▁www . ama zon . de ▁/ ▁ saw y ers - hu c kle berry"


In [18]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0, wd=0.1)

In [19]:
lr = 1e-3
lr *= bs/48

In [20]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.585116,4.181531,0.291311,05:05


In [21]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(5, slice(lr/10,lr*10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.390386,4.141454,0.297032,06:17
1,4.292165,4.066234,0.307350,06:18
2,4.131751,3.927355,0.321946,06:18
3,3.919436,3.777414,0.338433,06:18
4,3.735788,3.732159,0.343680,06:18


In [22]:
learn_lm.save(f'{lang}_csfd_2classes_fine_tuned')
learn_lm.save_encoder(f'{lang}_csfd_2classes_fine_tuned_enc')

### Classifier

In [74]:
data_clas = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='pos')
    .databunch(bs=bs // 2, num_workers=1))

In [96]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False, wd=0.1).to_fp16()
learn_c.load_encoder(f'{lang}_csfd_2classes_fine_tuned_enc')
learn_c.freeze()

In [76]:
lr = 2e-2
lr *= bs/48

In [77]:
learn_c.fit_one_cycle(3, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.307579,0.258988,0.892262,01:52
1,0.269991,0.248872,0.898532,01:36
2,0.274198,0.216400,0.914701,01:48


In [78]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(3, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.247607,0.206075,0.921135,02:12
1,0.215242,0.177407,0.928725,01:58
2,0.170189,0.169069,0.933674,01:56


In [79]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(3, slice(lr/4/(2.6**4),lr/4), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.220555,0.194989,0.919320,03:08
1,0.179532,0.158899,0.936809,03:50
2,0.107521,0.148248,0.944564,04:02


In [97]:
learn_c.save(f'{lang}_csfd_2classes');

In [ ]:
#learn_c.unfreeze()
#learn_c.fit_one_cycle(5, slice(lr/10/(2.6**4),lr/20), moms=(0.8,0.7))

In [ ]:
#learn_c.save(f'{lang}_csfd_2classes_overfitted')